In [79]:
from NSEDownload import stocks
import glob
import pandas as pd
import time
from random import randint
import os
import numpy as np

In [12]:
all_n50_scripts = pd.read_csv('data/nifty_50.csv')

This file contains NIFTY50 index as well. Removing it from the list.

In [13]:
all_n50_scripts = list(all_n50_scripts.Symbol)
all_n50_scripts = all_n50_scripts[1:]

In [14]:
completed_files = os.listdir('data\Adjusted')
completed_files = [file.split('.')[0] for file in completed_files if file.endswith('.csv')]

In [15]:
all_n50_scripts = [script for script in all_n50_scripts if script not in completed_files]
print(len(all_n50_scripts))

1


In [16]:
for script in all_n50_scripts:
    try:
        print("Runnning for {}".format(script))
        df = stocks.get_adjusted_stock(stockSymbol = script, start_date = '1-1-2000', end_date = '18-1-2022')
        df = df.reset_index()
        if df.shape[0] > 1:
            df.to_csv("data\Adjusted\{}.csv".format(script), index=False)
    except:
        print("Not successful for {}".format(script))
        time.sleep(randint(3,10))
        pass
    time.sleep(randint(3,10))

Runnning for EICHERMOT
EICHERMOT
SPLIT  on :  24-Aug-2020  and ratio is :  10.0


There are changes in symbol for few companies. Let's replace older symbol with latest one and merge all files into one.

In [17]:
file_path = 'data\Adjusted'
completed_files = os.listdir(file_path)
completed_files = [file for file in completed_files if file.endswith('.csv')]

In [21]:
all_scripts_list =[]
for file in completed_files:
    df = pd.read_csv(file_path + '\\' + file)
    if len(df['Symbol'].unique()) > 1:
        latest_symbol = df.tail(1)['Symbol'].values
        df['Symbol'] = latest_symbol[0]
    all_scripts_list.append(df)

In [22]:
all_scripts_df = pd.concat(all_scripts_list, axis=0, ignore_index=True)

This Data needs some cleaning before consumption. Next few steps will aim towards inspecting every column, changing data type(if required) and cleaning it.

In [23]:
all_scripts_df.Date = pd.to_datetime(all_scripts_df.Date)

In [24]:
all_scripts_df = all_scripts_df.drop(['Series'], axis=1)

In [25]:
all_scripts_df.columns

Index(['Date', 'Symbol', 'Open Price', 'High Price', 'Low Price', 'Last Price',
       'Close Price', 'Average Price', 'Total Traded Quantity', 'Turnover',
       'No. of Trades', 'Deliverable Qty', '% Dly Qt to Traded Qty', ' '],
      dtype='object')

In [26]:
all_scripts_df = all_scripts_df.rename(columns={'Open Price' : 'Open',
'Open Price' : 'Open',
'High Price' : 'High',
'Low Price' : 'Low',
'Last Price' : 'Last',
'Close Price' : 'Close',
'Average Price' : 'Average',
'Total Traded Quantity' : 'Quantity',
'No. of Trades' : 'Trades',
'Deliverable Qty' : 'Deliverable_quatity',
'% Dly Qt to Traded Qty' : 'Deliverable_quatity_pct',
})

In [27]:
all_scripts_df = all_scripts_df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Last', 'Close', 'Average',
       'Quantity', 'Turnover', 'Trades', 'Deliverable_quatity','Deliverable_quatity_pct']]

In [28]:
all_scripts_df.loc[all_scripts_df.Trades=='-','Trades'] = 0
all_scripts_df['Trades'] = all_scripts_df['Trades'].astype('int')

In [29]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity=='-','Deliverable_quatity'] = -1
all_scripts_df['Deliverable_quatity'] = all_scripts_df['Deliverable_quatity'].astype('int')

In [30]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity_pct=='-','Deliverable_quatity_pct'] = -1
all_scripts_df['Deliverable_quatity_pct'] = all_scripts_df['Deliverable_quatity_pct'].astype('float64')

In [31]:
all_scripts_df.loc[all_scripts_df.Trades==0, 'Trades'] = (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity'] * 100) / (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity_pct'] )

In [32]:
all_scripts_df = all_scripts_df.sort_values(by=['Symbol','Date'])

In [66]:
all_scripts_df['10d_max'] = all_scripts_df.groupby('Symbol').rolling(10)['Average'].max().groupby(level=0).shift(-10).droplevel(0)

In [70]:
all_scripts_df['10d_pct_change'] = (all_scripts_df['10d_max'] / all_scripts_df['Close'])-1

In [74]:
all_scripts_df = all_scripts_df.drop('10d_max', axis=1)

In [75]:
all_scripts_df.head()

,Date,Symbol,Open,High,Low,Last,Close,Average,Quantity,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct,10d_pct_change
0,2007-11-27,ADANIPORTS,154.00,210.00,154.0,191.8,192.58,196.944,27294366,2.687719e+10,2.729684e+07,9859619,36.12,0.149206
1,2007-11-28,ADANIPORTS,196.80,198.00,174.8,177.0,178.78,188.276,4581338,4.312765e+09,4.581583e+06,1453278,31.72,0.237913
2,2007-11-29,ADANIPORTS,181.80,182.95,168.2,177.4,176.84,177.618,5124121,4.550658e+09,5.122979e+06,1069678,20.88,0.254343
3,2007-11-30,ADANIPORTS,178.00,191.60,178.0,185.8,184.31,185.834,4609762,4.283257e+09,4.610285e+06,1260913,27.35,0.214855
4,2007-12-03,ADANIPORTS,187.95,199.00,184.4,196.0,193.86,193.130,2977470,2.875200e+09,2.977464e+06,816123,27.41,0.155009


In [ ]:
np.mean(abs(all_scripts_df['10d_pct_change']))

In [78]:
all_scripts_df['10d_pct_change'].describe()

count    239792.000000
mean          0.044233
std           0.057414
min          -0.926342
25%           0.007905
50%           0.030774
75%           0.065128
max           1.010024
Name: 10d_pct_change, dtype: float64

In [82]:
all_scripts_df.to_csv('data/n50_data.csv',index=False)